In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import speech_recognition as sr
import gradio as gr

# Ensure necessary NLTK data is available
nltk.download('punkt')
nltk.download('stopwords')

# Load CSV dataset safely
def load_dataset(file_path="cpcsv.csv"):
    try:
        df = pd.read_csv(file_path, on_bad_lines='skip', engine='python', encoding='utf-8')
        if 'query' not in df.columns or 'answer' not in df.columns:
            raise ValueError("CSV must contain 'query' and 'answer' columns.")
        return df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return pd.DataFrame(columns=['query', 'answer'])

# Load the data
data = load_dataset()

# Intent detection keywords
GREETINGS = ["hello", "hi", "hey", "greetings"]
GOODBYES = ["bye", "goodbye", "see you", "farewell"]

# Convert audio to text
def convert_speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_file) as source:
            audio = recognizer.record(source)
        return recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        return "Sorry, I couldn't understand the audio."
    except sr.RequestError:
        return "Speech service is unavailable. Please try again later."

# Get chatbot response
def get_response(user_input):
    if data.empty:
        return "Dataset is not loaded. Please check the CSV file."

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(user_input.lower())
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    if any(word in tokens for word in GREETINGS):
        return "Hello! How can I help you with child psychology today?"
    if any(word in tokens for word in GOODBYES):
        return "Goodbye! Take care and feel free to come back anytime."

    # Match based on content words
    for _, row in data.iterrows():
        question_tokens = word_tokenize(row['query'].lower())
        question_filtered = [word for word in question_tokens if word.isalpha() and word not in stop_words]
        if set(filtered_tokens) & set(question_filtered):
            return row['answer']

    return "I'm not sure how to answer that. You might want to consult a psychologist."

# Interface function
def chatbot_interface(text_input, audio_input):
    if audio_input is not None:
        user_query = convert_speech_to_text(audio_input)
    else:
        user_query = text_input

    if not user_query.strip():
        return "Please enter or say something."

    return get_response(user_query)

# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 👶 Child Psychology Chatbot")

    with gr.Row():
        text_box = gr.Textbox(label="Type your question here")
        audio_upload = gr.Audio(type="filepath", label="Or upload an audio file")

    output_box = gr.Textbox(label="Chatbot's Response")

    text_box.submit(chatbot_interface, inputs=[text_box, audio_upload], outputs=output_box)
    audio_upload.change(chatbot_interface, inputs=[text_box, audio_upload], outputs=output_box)

    gr.Markdown("Upload your query as audio or type it above to receive a response.")

app.launch(share=True, debug=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a04796dc49e6020770.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving cpcsv.csv to cpcsv (1).csv


In [ ]:
import os
print(os.listdir())


['.config', 'cpcsv.csv', 'cpcsv (1).csv', '.gradio', 'sample_data']


In [ ]:
import os
csv_path = os.path.abspath("cpcsv.csv")
print(csv_path)


/content/cpcsv.csv


In [ ]:
import pandas as pd
df = pd.read_csv("cpcsv.csv")
print(df.head())


                                               query  \
0  What resources are available for children with...   
1            Are there any therapies for Depression?   
2  Can Behavioral issues affect my child's develo...   
3  How can I support a child with Anxiety at school?   
4         Can Anxiety affect my child's development?   

                                              answer  
0  Depression in children can manifest as sadness...  
1  Depression in children can manifest as sadness...  
2  Behavioral issues in children can include defi...  
3  Children with anxiety may experience excessive...  
4  Children with anxiety may experience excessive...  


In [ ]:
import gradio as gr

def chatbot_interface(file):
    import pandas as pd
    import io
    df = pd.read_csv(io.BytesIO(file.read()))
    # Process the DataFrame as needed
    return df.head().to_string()

gr.Interface(fn=chatbot_interface, inputs=gr.File(), outputs="text").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9508f1e054e8c73730.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
pip install pandas

In [ ]:
pip install gradio

In [ ]:
pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 22.4 MB/s eta 0:00:00


In [ ]:
pip install nltk